## BRONZE TO SILVER LAYER

### Silver Layer - Stock Price History


In [1]:
# Importing Common Utility Funcation

import re

import pandas as pd
from StockETL import GlobalPath

In [2]:
# Import necessary libraries and utility functions
%run ../COMMON/common_utility.ipynb

In [3]:
# Instantiate GlobalPath
stockdata_bronze_layer_path = GlobalPath("DATA/BRONZE/StockData")
stockprice_silver_file_path = GlobalPath(
    "DATA/SILVER/StockPrice/StockPrice_data.csv"
)

### Data Processing

- Initialize an empty list to store DataFrames.
- Read and concatenate data from multiple CSV files.


In [4]:
# Initialize an empty list to store individual stock price DataFrames
df_stock_price_list = []

# Generate file paths for available CSV files in the Bronze layer
file_paths = check_files_availability(
    stockdata_bronze_layer_path, file_pattern="*.csv"
)

# Loop through the list of CSV files in the folder
for file_path in file_paths:
    print(f"Processing file: {file_path}")

    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Extract the stock symbol by removing the date pattern from the file name
    df["symbol"] = re.sub(r"_\d{4}_\d{2}\.csv", "", file_path.name)

    # Append the DataFrame to the list of DataFrames
    df_stock_price_list.append(df)

# Concatenate all individual DataFrames into one combined DataFrame
df = pd.concat(df_stock_price_list, ignore_index=True)

# Standardize column names by replacing punctuation
df = replace_punctuation_from_columns(df)

# Remove columns that contain only NA values
df.dropna(how="all", axis=1, inplace=True)

Number of Files Detected: 1170
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\AWL_2022_02.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\AWL_2022_03.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\AWL_2022_04.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\AWL_2022_05.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\AWL_2022_06.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\AWL_2022_07.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\AWL_2022_08.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\AWL_2022_09.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\AWL_2022_10.csv
Processing file: C:\Users\prashan

Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\DEEPAKNTR_2024_08.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\DEEPAKNTR_2024_09.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\FCSSOFT_2022_01.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\FCSSOFT_2022_02.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\FCSSOFT_2022_03.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\FCSSOFT_2022_04.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\FCSSOFT_2022_05.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\FCSSOFT_2022_06.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\FCSSOFT_2022_07.csv
Processing file: C:\User

Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\HFCL_2023_03.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\HFCL_2023_04.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\HFCL_2023_05.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\HFCL_2023_06.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\HFCL_2023_07.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\HFCL_2023_08.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\HFCL_2023_09.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\HFCL_2023_10.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\HFCL_2023_11.csv
Processing file: C:\Users\prashant.tripathi\Code\Portfo

Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\LLOYDSENGG_2022_01.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\LLOYDSENGG_2022_02.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\LLOYDSENGG_2022_03.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\LLOYDSENGG_2022_04.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\LLOYDSENGG_2022_05.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\LLOYDSENGG_2022_06.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\LLOYDSENGG_2022_07.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\LLOYDSENGG_2022_08.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\LLOYDSENGG_2022_09.csv
P

Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\RPOWER_2024_06.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\RPOWER_2024_07.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\RPOWER_2024_08.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\RPOWER_2024_09.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\RTNINDIA_2021_12.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\RTNINDIA_2022_01.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\RTNINDIA_2022_02.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\RTNINDIA_2022_03.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\RTNINDIA_2022_04.csv
Processing file: C:\Users\p

Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\TATACHEM_2021_11.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\TATACHEM_2021_12.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\TATACHEM_2022_01.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\TATACHEM_2022_02.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\TATACHEM_2022_03.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\TATACHEM_2022_04.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\TATACHEM_2022_05.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\TATACHEM_2022_06.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\TATACHEM_2022_07.csv
Processing file: C:

Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\TRIDENT_2022_07.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\TRIDENT_2022_08.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\TRIDENT_2022_09.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\TRIDENT_2022_10.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\TRIDENT_2022_11.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\TRIDENT_2022_12.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\TRIDENT_2023_01.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\TRIDENT_2023_02.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\TRIDENT_2023_03.csv
Processing file: C:\Users\pr

Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\VIJIFIN_2023_08.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\VIJIFIN_2023_09.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\VIJIFIN_2023_10.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\VIJIFIN_2023_11.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\VIJIFIN_2023_12.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\VIJIFIN_2024_01.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\VIJIFIN_2024_02.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\VIJIFIN_2024_03.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\VIJIFIN_2024_04.csv
Processing file: C:\Users\pr

Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\VOLTAS_2022_05.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\VOLTAS_2022_06.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\VOLTAS_2022_07.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\VOLTAS_2022_08.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\WIPRO_2022_01.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\WIPRO_2022_02.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\WIPRO_2022_03.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\WIPRO_2022_04.csv
Processing file: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\StockData\WIPRO_2022_05.csv
Processing file: C:\Users\prashant.tripath

### Final Processing and Export

- Round numerical values to 2 decimal places.
- Sort the DataFrame by stock name and date.
- Save the processed data as a CSV file in the Silver layer.


In [5]:
# Convert datetime to date string
df["date"] = pd.to_datetime(df["date"]).dt.date

# Round numerical values to 2 decimal places
df = df.round(2)

# Sort the DataFrame by stock name and date
df = df.sort_values(by=["symbol", "date"])

# Select relevant columns
df = df[
    [
        "date",
        "symbol",
        "open",
        "high",
        "low",
        "close",
        "volume",
    ]
]

# Save the result as a CSV file
df.to_csv(stockprice_silver_file_path, index=None)
print("SILVER Layer CSV file for Stock Price history successfully created at:")
print(stockprice_silver_file_path)
# Log the DataFrame debug
df.info()

SILVER Layer CSV file for Stock Price history successfully created at:
C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\SILVER\StockPrice\StockPrice_data.csv
<class 'pandas.core.frame.DataFrame'>
Index: 22788 entries, 0 to 22787
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    22788 non-null  object 
 1   symbol  22788 non-null  object 
 2   open    22788 non-null  float64
 3   high    22788 non-null  float64
 4   low     22788 non-null  float64
 5   close   22788 non-null  float64
 6   volume  22788 non-null  float64
dtypes: float64(5), object(2)
memory usage: 1.4+ MB
